In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 17 21:49:30 2022

@author: HP
"""

import numpy as np
import pandas as pd
import random
import math
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

# This function evaluate the weighted distance of customers to DCs
def evaldist(individual):
    # Assign customers to DCs randomly
    c2DC = np.zeros((customN,DCN), dtype=np.uint8)
    c2DC[np.arange(customN) , np.random.randint(0,DCN, size=customN)] = 1
    # Calculate the weighted distance
    for j in range(DCN):
        Dist[j] = 0
        for i in range(customN):
            if c2DC[i,j] == 1:
                Dist[j] += math.sqrt((individual[j*2]-df[i,0])**2+(individual[j*2+1]-df[i,1])**2)*df[i,2]
    return sum(Dist),

df = pd.read_excel('C:/Users/HP/Desktop/GFA.xlsx' )
df = df.to_numpy()
customN = (len(df))
DCN = int(input("Enter the number of distribution centers?"))
indLength = DCN*2
Dist = np.empty(shape=(DCN))
minvalue = df[:, 0:2].min()
maxvalue = df[:, 0:2].max()
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_xy", random.randint, minvalue, maxvalue)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_xy, indLength)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaldist)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, 
                               halloffame=hof, verbose=True)
print(f"The best individual: {hof}")
